# 简述

参考：
https://zhuanlan.zhihu.com/p/452972774
https://www.jianshu.com/p/733501e64966
https://blog.csdn.net/pearl8899/article/details/107936792

In [2]:
person_data = {"name": ["Abby","Bella","Cathy","Dell"],
               "gender":[0,0,0,1],
               "degree":[0,1,2,3],
               "age":[38,27,24,18],
               "salary":[1000.8,900.7,1200,400],
               "pet":[["dog","starfish"],["cat","fish"],["tiger","cat"],["dog","cat"]],
               "province":["安徽省","上海市","北京市","新疆维吾尔自治区"]
               }

province_list = ['北京市','上海市','天津市','重庆市','河北省','山西省','内蒙古自治区','辽宁省','吉林省','黑龙江省',
 '江苏省','浙江省','安徽省','福建省','江西省','山东省','河南省','湖北省','湖南省','广东省','广西壮族自治区','海南省',
 '四川省','贵州省','云南省','西藏自治区','陕西省','甘肃省','青海省','宁夏回族自治区','新疆维吾尔自治区','台湾省','香港特别行政区', '澳门特别行政区']

person_data

{'age': [38, 27, 24, 18],
 'degree': [0, 1, 2, 3],
 'gender': [0, 0, 0, 1],
 'name': ['Abby', 'Bella', 'Cathy', 'Dell'],
 'pet': [['dog', 'starfish'],
  ['cat', 'fish'],
  ['tiger', 'cat'],
  ['dog', 'cat']],
 'province': ['安徽省', '上海市', '北京市', '新疆维吾尔自治区'],
 'salary': [1000.8, 900.7, 1200, 400]}

针对person_data数据，

In [54]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow.python.ops import lookup_ops


# 整数-> OneHot
gender_column = tf.feature_column.indicator_column(
tf.feature_column.categorical_column_with_identity("gender",2))

# Hash-> MultiHot
pet_column = tf.feature_column.indicator_column(
tf.feature_column.categorical_column_with_hash_bucket("pet",5))

# 词表-> OneHot
province_column = tf.feature_column.indicator_column(
tf.feature_column.categorical_column_with_vocabulary_list("province",vocabulary_list=province_list,num_oov_buckets=1))

# Embedding
name_column = tf.feature_column.embedding_column(
tf.feature_column.categorical_column_with_hash_bucket("name",10),10
)

# Dense特征
salary_column = tf.feature_column.numeric_column("salary",normalizer_fn=lambda x: (x - 800) / 100)

# Dense特征
age_column = tf.feature_column.bucketized_column(tf.feature_column.numeric_column("age"),boundaries=[20, 30])

# Cross特征
age_degree_column = tf.feature_column.embedding_column(
tf.feature_column.crossed_column([tf.feature_column.categorical_column_with_identity("gender",2),tf.feature_column.categorical_column_with_identity("degree",4)],hash_bucket_size=8),8
)

# 构建input_fn函数
gender_input_fn = tf.feature_column.input_layer(person_data, gender_column)
pet_input_fn = tf.feature_column.input_layer(person_data, pet_column)
name_input_fn = tf.feature_column.input_layer(person_data, name_column)
province_input_fn = tf.feature_column.input_layer(person_data, province_column)
salary_input_fn = tf.feature_column.input_layer(person_data, salary_column)
age_input_fn = tf.feature_column.input_layer(person_data, age_column)
age_degree_input_fn =  tf.feature_column.input_layer(person_data, age_degree_column)

# 完整input_fn函数
all_input_fn = tf.feature_column.input_layer(person_data, [gender_column,pet_column,name_column,province_column,salary_column,age_column,age_degree_column])

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(lookup_ops.tables_initializer())
    print(all_input_fn.eval())

[[ 0.          0.          1.          0.09848841  0.15381086 -0.23387277
  -0.61514837 -0.20832168 -0.08198683 -0.3471508  -0.46080923  1.
   0.          0.18147144  0.34135872 -0.03115987 -0.21193358  0.42701536
  -0.57190907  0.12232681  0.10418347 -0.17313461  0.24740234  0.
   0.          1.          0.          1.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          1.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.
   0.          0.          0.          2.008     ]
 [ 0.          1.          0.         -0.13640153 -0.3423585  -0.41799852
  -0.00921592 -0.00841665  0.07088871 -0.16311663  0.08244163  1.
   0.          0.1758039   0.02773052 -0.17290676  0.05382605  0.07488564
  -0.08381433  0.05588934  0.60138625 -0.12724176 -0.269000